## Задача 1
С помощью SQL запроса подготовьте данные для оценки рекламных кампаний. Итоговый отчет должен содержать сводные показатели, сгруппированные по дате, каналу и кампании привлечения пользователей.

In [2]:
import pandas as pd
import sqlite3

In [3]:
#Читаем CSV-файлы

users = pd.read_csv("users.csv")
attribution = pd.read_csv("attribution.csv")
payments = pd.read_csv("payments.csv")
ad_data = pd.read_csv("ad_data.csv")

In [4]:
#Создаём базу данных в памяти

conn = sqlite3.connect(":memory:")

In [5]:
#Загружаем таблицы в базу

users.to_sql("users", conn, index=False, if_exists="replace")
attribution.to_sql("attribution", conn, index=False, if_exists="replace")
payments.to_sql("payments", conn, index=False, if_exists="replace")
ad_data.to_sql("ad_data", conn, index=False, if_exists="replace")

91

In [6]:
#Функция для SQL-запросов

def sql(query):
    return pd.read_sql_query(query, conn)

Проверка users:
- нет ли дубликатов
- нет ли NULL
- даты в разумном диапазоне

In [7]:
# количество уникальных и общих user_id в users

sql("""
SELECT COUNT(DISTINCT user_id) AS unique_users,
       COUNT(*) AS total_rows
FROM users;
""")


,unique_users,total_rows
0,29767,29767


In [8]:
# пустые значения в users

sql("""
SELECT *
FROM users
WHERE user_id IS NULL
   OR event_datetime IS NULL;
""")

,event_datetime,user_id,platform,country


In [9]:
# крайние даты установок в users

sql("""
SELECT MIN(event_datetime) AS min_date,
       MAX(event_datetime) AS max_date
FROM users;
""")

,min_date,max_date
0,2024-03-01 00:04:31,2024-03-31 23:54:55


Выводы по users:

- 29 767 уникальных пользователей
- нет дубликатов и пропусков
- даты установок от 1 до 31 марта 2024

Проверка attribution:

- нет ли user_id с несколькими каналами/кампаниями
- нет ли NULL в user_id, media_source, campaign
- все ли user_id есть в users

In [10]:
# уникальность user_id в attribution

sql("""
SELECT COUNT(DISTINCT user_id) AS unique_users,
       COUNT(*) AS total_rows
FROM attribution;
""")


,unique_users,total_rows
0,20711,20711


In [11]:
# пустые значения в attribution

sql("""
SELECT *
FROM attribution
WHERE user_id IS NULL
   OR media_source IS NULL
   OR campaign IS NULL;
""")

,user_id,media_source,campaign


In [12]:
# пользователи в attribution, которых нет в users

sql("""
SELECT attribution.user_id
FROM attribution
LEFT JOIN users ON attribution.user_id = users.user_id
WHERE users.user_id IS NULL;
""")

,user_id


Выводы по attribution:

- 20 711 строк, 20 711 уникальных пользователей
- нет дубликатов
- нет пропусков в user_id, media_source, campaign
- все user_id присутствуют в users

Проверка payments:

- нет ли NULL в user_id и amount
- нет ли отрицательных значений amount
- все ли user_id есть в users
- нет ли платежей раньше даты установки пользователя

In [13]:
# Проверка на NULL в ключевых полях в Payments

sql("""
SELECT COUNT(*) AS bad_rows
FROM payments
WHERE user_id IS NULL
   OR revenue IS NULL
   OR event_datetime IS NULL;
""")

,bad_rows
0,0


In [14]:
# Проверка на отрицательные платежи в Payments

sql("""
SELECT COUNT(*) AS negative_revenue
FROM payments
WHERE revenue < 0;
""")

,negative_revenue
0,0


In [15]:
# Проверка что все user_id есть в users в Payments

sql("""
SELECT COUNT(*) AS missing_users
FROM payments
LEFT JOIN users ON payments.user_id = users.user_id
WHERE users.user_id IS NULL;
""")

,missing_users
0,0


In [16]:
# Проверка нет ли платежей раньше даты установки пользователя в Payments

sql("""
SELECT COUNT(*) AS payments_before_install
FROM payments
JOIN users ON payments.user_id = users.user_id
WHERE payments.event_datetime < users.event_datetime;
""")


,payments_before_install
0,0


Выводы по payments:

- нет NULL в user_id, event_datetime, revenue
- нет отрицательных значений revenue
- все user_id присутствуют в users
- нет платежей раньше даты установки

Проверка ad_data:

- нет ли NULL в date, media_source, campaign, costs, impressions, clicks
- нет ли отрицательных значений в costs, impressions, clicks
- даты в разумном диапазоне (совпадают с периодом в users/payments)
- нет ли дублей по ключу: date + media_source + campaign + platform + country
- логика метрик: clicks <= impressions (CTR не > 100%)

In [17]:
# Проверка NULL в ad_data

sql("""
SELECT COUNT(*) AS bad_rows
FROM ad_data
WHERE date IS NULL
   OR media_source IS NULL
   OR campaign IS NULL
   OR costs IS NULL
   OR impressions IS NULL
   OR clicks IS NULL;
""")


,bad_rows
0,0


In [18]:
# Проверка отрицательных значений в ad_data

sql("""
SELECT COUNT(*) AS negative_values
FROM ad_data
WHERE costs < 0
   OR impressions < 0
   OR clicks < 0;
""")


,negative_values
0,0


In [19]:
# Проверка диапазона дат в ad_data

sql("""
SELECT MIN(date) AS min_date,
       MAX(date) AS max_date
FROM ad_data;
""")


,min_date,max_date
0,2024-03-01,2024-03-31


In [20]:
# Проверка дублей по ключу

sql("""
SELECT COUNT(*) AS duplicate_groups
FROM (
    SELECT 
        date, 
        media_source, 
        campaign, 
        platform, 
        country, 
        COUNT(*) AS rows_in_group
    FROM ad_data
    GROUP BY date, media_source, campaign, platform, country
    HAVING COUNT(*) > 1
) AS grouped_combinations;
""")


,duplicate_groups
0,0


In [21]:
# Проверка: кликов не больше, чем показов

sql("""
SELECT COUNT(*) AS invalid_ctr_rows
FROM ad_data
WHERE clicks > impressions;
""")


,invalid_ctr_rows
0,0


Выводы по ad_data:

- нет NULL в date, media_source, campaign, costs, impressions, clicks
- нет отрицательных значений в costs, impressions, clicks
- даты в разумном диапазоне, совпадают с периодом users и payments
- нет дублей по ключу date + media_source + campaign + platform + country
- clicks не превышают impressions (CTR ≤ 100%)

In [22]:
sql("""
SELECT COUNT(*) AS unique_combinations
FROM (
    SELECT DISTINCT campaign
    FROM ad_data
);
""")


,unique_combinations
0,6


Подготовка сводной таблицы:
- сгруппировать данные по date, media_source, campaign
- из users + attribution получить количество установок (installs)
- из payments + attribution получить выручку (revenue)
- из ad_data получить расходы (costs), показы (impressions) и клики (clicks)
- объединить всё по ключу date + media_source + campaign

Каркас из ad_data: дневные рекламные метрики
Агрегируем расходы, показы и клики по ключу date + media_source + campaign.

In [23]:
ad_metrics = sql("""
SELECT
  DATE(date)       AS date,
  media_source,
  campaign,
  SUM(costs)       AS costs,
  SUM(impressions) AS impressions,
  SUM(clicks)      AS clicks
FROM ad_data
GROUP BY DATE(date), media_source, campaign
ORDER BY date, media_source, campaign;
""")
ad_metrics.head()


,date,media_source,campaign,costs,impressions,clicks
0,2024-03-01,tiktokglobal_int,tt_campaign_1,373.682987,55620,2201
1,2024-03-01,tiktokglobal_int,tt_campaign_4,130.908847,27650,675
2,2024-03-02,tiktokglobal_int,tt_campaign_1,370.283288,85610,1470
3,2024-03-02,tiktokglobal_int,tt_campaign_4,123.301634,17787,494
4,2024-03-03,tiktokglobal_int,tt_campaign_1,373.219923,74222,2512


Установки (installs) из users + attribution
Берём дату установки из users.event_datetime, а привязку к каналу/кампании — из attribution по user_id. Считаем уникальных пользователей.

In [24]:
daily_installs = sql("""
SELECT
  DATE(users.event_datetime)    AS date,
  attribution.media_source      AS media_source,
  attribution.campaign          AS campaign,
  COUNT(DISTINCT users.user_id) AS installs
FROM users
JOIN attribution
  ON users.user_id = attribution.user_id
GROUP BY DATE(users.event_datetime), attribution.media_source, attribution.campaign
ORDER BY date, media_source, campaign;
""")
daily_installs.head()


,date,media_source,campaign,installs
0,2024-03-01,tiktokglobal_int,tt_campaign_1,254
1,2024-03-01,tiktokglobal_int,tt_campaign_4,93
2,2024-03-02,tiktokglobal_int,tt_campaign_1,218
3,2024-03-02,tiktokglobal_int,tt_campaign_4,60
4,2024-03-03,tiktokglobal_int,tt_campaign_1,405


Выручка (revenue) из payments + attribution
Берём дату платежа из payments.event_datetime, суммируем revenue, привязываем к каналу/кампании через attribution.

In [25]:
daily_revenue = sql("""
SELECT
  DATE(payments.event_datetime) AS date,
  attribution.media_source      AS media_source,
  attribution.campaign          AS campaign,
  SUM(payments.revenue)         AS revenue
FROM payments
JOIN attribution
  ON payments.user_id = attribution.user_id
GROUP BY DATE(payments.event_datetime), attribution.media_source, attribution.campaign
ORDER BY date, media_source, campaign;
""")
daily_revenue.head()


,date,media_source,campaign,revenue
0,2024-03-02,tiktokglobal_int,tt_campaign_1,11.823100
1,2024-03-02,tiktokglobal_int,tt_campaign_4,2.350400
2,2024-03-03,tiktokglobal_int,tt_campaign_1,58.466801
3,2024-03-03,tiktokglobal_int,tt_campaign_4,38.486001
4,2024-03-04,applovin_int,al_campaign_1,9.737900


Финальная сводная таблица
К «каркасу» из ad_data подтягиваем installs и revenue по ключу date + media_source + campaign. Там, где значений нет, ставим 0.

In [26]:
report = sql("""
SELECT
  ad.date,
  ad.media_source,
  ad.campaign,
  COALESCE(di.installs, 0) AS installs,
  COALESCE(dr.revenue, 0)  AS revenue,
  ad.costs,
  ad.impressions,
  ad.clicks
FROM (
  SELECT
    DATE(date)       AS date,
    media_source,
    campaign,
    SUM(costs)       AS costs,
    SUM(impressions) AS impressions,
    SUM(clicks)      AS clicks
  FROM ad_data
  GROUP BY DATE(date), media_source, campaign
) AS ad
LEFT JOIN (
  SELECT
    DATE(users.event_datetime)    AS date,
    attribution.media_source      AS media_source,
    attribution.campaign          AS campaign,
    COUNT(DISTINCT users.user_id) AS installs
  FROM users
  JOIN attribution
    ON users.user_id = attribution.user_id
  GROUP BY DATE(users.event_datetime), attribution.media_source, attribution.campaign
) AS di
  ON di.date = ad.date
 AND di.media_source = ad.media_source
 AND di.campaign = ad.campaign
LEFT JOIN (
  SELECT
    DATE(payments.event_datetime) AS date,
    attribution.media_source      AS media_source,
    attribution.campaign          AS campaign,
    SUM(payments.revenue)         AS revenue
  FROM payments
  JOIN attribution
    ON payments.user_id = attribution.user_id
  GROUP BY DATE(payments.event_datetime), attribution.media_source, attribution.campaign
) AS dr
  ON dr.date = ad.date
 AND dr.media_source = ad.media_source
 AND dr.campaign = ad.campaign
ORDER BY ad.date, ad.media_source, ad.campaign;
""")
report.head()


,date,media_source,campaign,installs,revenue,costs,impressions,clicks
0,2024-03-01,tiktokglobal_int,tt_campaign_1,254,0.000000,373.682987,55620,2201
1,2024-03-01,tiktokglobal_int,tt_campaign_4,93,0.000000,130.908847,27650,675
2,2024-03-02,tiktokglobal_int,tt_campaign_1,218,11.823100,370.283288,85610,1470
3,2024-03-02,tiktokglobal_int,tt_campaign_4,60,2.350400,123.301634,17787,494
4,2024-03-03,tiktokglobal_int,tt_campaign_1,405,58.466801,373.219923,74222,2512


In [27]:
report.to_csv("marketing_summary_by_date_channel_campaign.csv", index=False)


## Вывод по сводной таблице

- данные сгруппированы по date + media_source + campaign
- объединены расходы (costs, impressions, clicks) из ad_data
- добавлены установки (installs) из users + attribution
- добавлена выручка (revenue) из payments + attribution
- органические пользователи не включены (учтены только атрибутированные)
- пропуски заменены на 0 с помощью COALESCE